# fMRI Analysis 2018
For single scans (have to be in .BRIK format already). Needs AFNI installed.

**Usage**: Adjust the settings below.<br> 
Run this script ("*Run*" button above, or CTRL+ENTER to run cell-by-cell). <br>
Popup will open --> select the scan. <br>
Results are stored in ./analyze subfolder

## 1. Important settings

In [1]:
TR = 1 # in seconds

initial_cutoff = 30 # number of volumes to remove (in absence of dummy scans).

blurFWHM = 0.3 # In [mm]. Rule-of-thumb: 1.5x the voxel resolution.

highpass = 9999 # period [s]. Any signal slower than this will be removed. Should be at least 2x inter-stim. interval


# --- For averaging of blocks:

averaging_baseline = 20 # baseline to consider before each block

_time = 60 # how many seconds to display after stim block ends


# --- Location of an EPI template for coregistration (should be a representative animal with same geometry settings)

template_path = '/home/aic/BigHDD/MR_DATA/CSI2018/template/TEMPLATE+orig'


# --- Define the stimulation paradigm:

baseline = 12*60
stim_duration = 40  # everything in seconds
Nblocks = 10
ISI = 60  # Inter-stimulus interval (on+off period)

# --- ROI file:
roi_path = '/home/aic/BigHDD/MR_DATA/CSI2018/template/ROI+orig'




#----
stimList=list(range(baseline,baseline+ISI*Nblocks,ISI))
stimString=' '.join(map(str,stimList))
stimulus_times = "'1D: "+stimString+"'"
print('stimulus times:',stimList)

stimulus times: [720, 780, 840, 900, 960, 1020, 1080, 1140, 1200, 1260]


In [2]:
import os
import subprocess
import shutil
import sys
import numpy as np
import glob
from tkinter import Tk
from tkinter.filedialog import askopenfilename

## 2. Selecting scan

In [3]:
if 'file_path' not in vars():

    root = Tk()
    ftypes = [('EPI Scan',"*.BRIK*"),("All Files","*.*")]
    ttl = "Select file"
    dir1 = '/home/aic/BigHDD/MR_DATA/CSI2018'

    root.withdraw()
    file_path = askopenfilename(filetypes = ftypes, initialdir = dir1, title = ttl)
    root.update()
    root.destroy()
    
        
    print('loaded '+file_path)

if 'ID' not in vars():
    ID = scanname
    
scanname=os.path.splitext(os.path.basename(file_path))[0][:-5]

    
data_path = os.path.dirname(file_path)
new_subfolder = 'analyzed'
newfolder = os.path.join(os.path.expanduser(data_path),new_subfolder)
os.chdir(data_path)
shutil.rmtree(newfolder,ignore_errors=True)


loaded /BigHDD/MR_DATA/CSI2018/20180620_133404_CSI_BOLD_Elec_Stim_1mA_1_4/8/pdata/1/E8+orig.BRIK


In [4]:
def runAFNI(inputstring):
    print(inputstring)
    subprocess.run(inputstring,shell=True, check=True)

## 3. Coregistration & Motion correction

In [5]:
runAFNI('3dTcat -prefix '+ newfolder + '/' + scanname + ' ' + scanname + "+orig'["+ str(initial_cutoff) + "..$]'")
runAFNI('3dTcat -prefix '+ newfolder + '/single_timepoint' + ' ' + scanname + "+orig'[0]'")

os.chdir(newfolder)

runAFNI("python2.7 /home/aic/abin/align_epi_anat.py -dset2to1 -dset1 " + template_path + " -dset2 single_timepoint+orig" \
                                " -child_dset2 " + scanname + '+orig ' \
                                " -dset1_strip None -dset2_strip None"  \
                                " -overwrite" \
                                " -big_move" \
                                " -volreg on -volreg_method 3dvolreg" \
                                " -volreg_opts '-Fourier -zpad 1'" \
                                " -Allineate_opts '-maxrot 10 -maxshf 3 -conv 0.005 -twofirst -twoblur 0.8 -source_automask+2 -final wsinc5'" \
                                " -tshift on -tshift_opts '-tzero 0 -quintic'" \
                                " -dset2_base 0" \
                                " -volreg_base 0" \
                                " -suffix .volreg" \
                                " -cost ls")
    
runAFNI('3dTstat -prefix rm.mean' + ' ' + scanname + '.volreg+orig')
runAFNI('3dcalc -a ' + scanname + '.volreg+orig -b rm.mean+orig'\
                    " -expr 'min(200, a/b*100)*step(a)*step(b)' -prefix r" + scanname + '.scale')

3dTcat -prefix /BigHDD/MR_DATA/CSI2018/20180620_133404_CSI_BOLD_Elec_Stim_1mA_1_4/8/pdata/1/analyzed/E8 E8+orig'[30..$]'
3dTcat -prefix /BigHDD/MR_DATA/CSI2018/20180620_133404_CSI_BOLD_Elec_Stim_1mA_1_4/8/pdata/1/analyzed/single_timepoint E8+orig'[0]'
python2.7 /home/aic/abin/align_epi_anat.py -dset2to1 -dset1 /home/aic/BigHDD/MR_DATA/CSI2018/template/TEMPLATE+orig -dset2 single_timepoint+orig -child_dset2 E8+orig  -dset1_strip None -dset2_strip None -overwrite -big_move -volreg on -volreg_method 3dvolreg -volreg_opts '-Fourier -zpad 1' -Allineate_opts '-maxrot 10 -maxshf 3 -conv 0.005 -twofirst -twoblur 0.8 -source_automask+2 -final wsinc5' -tshift on -tshift_opts '-tzero 0 -quintic' -dset2_base 0 -volreg_base 0 -suffix .volreg -cost ls
3dTstat -prefix rm.mean E8.volreg+orig
3dcalc -a E8.volreg+orig -b rm.mean+orig -expr 'min(200, a/b*100)*step(a)*step(b)' -prefix rE8.scale


## 4. Preprocessing
Detrending/Filtering/Blurring

In [6]:
# runAFNI('3dTproject -input r*.scale+orig.BRIK ' + '-CENSORTR ' + str(baseline)+':'+str(baseline+ISI*Nblocks) + ' -passband ' + str(1/highpass) + ' 99999 -blur ' + str(blurFWHM) + ' -prefix CleanedData')
runAFNI('3dTproject -input r*.scale+orig.BRIK -polort -1 -blur ' + str(blurFWHM) + ' -prefix CleanedData'+ ID)

3dTproject -input r*.scale+orig.BRIK -CENSORTR 720:1920 -polort 1 -blur 0.3 -prefix CleanedDataE8


## 5. GLM analysis

In [7]:
runAFNI('3dDeconvolve -input CleanedData'+ID+'+orig' \
                    ' -polort 3' \
                    ' -nodmbase' \
                    ' -overwrite' \
                    ' -num_stimts 1' \
                    ' -stim_times 1 ' +  stimulus_times + \
                    " 'SPMG3(" + str(stim_duration) + ")'" \
                    ' -stim_times_subtract ' + str(initial_cutoff*TR) + \
                    ' -stim_label 1 electro' \
                    ' -iresp 1 HRF_SPM_el' \
                    ' -fout -tout -x1D XSPM.xmat.1D -xjpeg XSPM.jpg' \
                    ' -fitts fittsSPM_'+ ID + \
                    ' -errts errtsSPM_el' \
                    ' -bucket statsSPM_'+ ID + \
                    ' -cbucket regcoeffsSPM_el')

3dDeconvolve -input CleanedDataE8+orig -polort -1 -nodmbase -overwrite -num_stimts 1 -stim_times 1 '1D: 720 780 840 900 960 1020 1080 1140 1200 1260' 'SPMG3(40)' -stim_times_subtract 30 -stim_label 1 electro -iresp 1 HRF_SPM_el -fout -tout -x1D XSPM.xmat.1D -xjpeg XSPM.jpg -fitts fittsSPM_E8 -errts errtsSPM_el -bucket statsSPM_E8 -cbucket regcoeffsSPM_el


In [ ]:
runAFNI('3dSynthesize -cbucket regcoeffsSPM_el+orig -matrix XSPM.xmat.1D -select baseline -prefix glmbaseline')

runAFNI("3dcalc -a CleanedData*.BRIK -b glmbaseline+orig -expr 'a-b' -prefix CleanedData2"+ID) 
runAFNI("3dcalc -a fittsSPM*.BRIK -b glmbaseline+orig -expr 'a-b' -prefix Cleanedfit") 

## 6. Averaging stimulation blocks
For plotting 

In [8]:
stimList

[720, 780, 840, 900, 960, 1020, 1080, 1140, 1200, 1260]

In [9]:
k=1
for i in stimList:
    i -= initial_cutoff
    runAFNI('3dTcat -prefix temp_average_raw' + str(k) + ' CleanedData'+ID+'+orig[' + str(int(i) - averaging_baseline ) + '..' + str(int(i) + stim_duration + _time) +']')
    k += 1

    
runAFNI('3dTcat -prefix average_spm' + ' fittsSPM_' + ID + '+orig[' + str(stimList[0] - averaging_baseline ) + '..' + str(stimList[0] + stim_duration + _time) +']')
runAFNI('3dMean -prefix average_raw temp_average_raw*')

runAFNI("bash -c 'rm -f temp_average* &>/dev/null'")


3dTcat -prefix temp_average_raw1 CleanedDataE8+orig[670..790]
3dTcat -prefix temp_average_raw2 CleanedDataE8+orig[730..850]
3dTcat -prefix temp_average_raw3 CleanedDataE8+orig[790..910]
3dTcat -prefix temp_average_raw4 CleanedDataE8+orig[850..970]
3dTcat -prefix temp_average_raw5 CleanedDataE8+orig[910..1030]
3dTcat -prefix temp_average_raw6 CleanedDataE8+orig[970..1090]
3dTcat -prefix temp_average_raw7 CleanedDataE8+orig[1030..1150]
3dTcat -prefix temp_average_raw8 CleanedDataE8+orig[1090..1210]
3dTcat -prefix temp_average_raw9 CleanedDataE8+orig[1150..1270]
3dTcat -prefix temp_average_raw10 CleanedDataE8+orig[1210..1330]
3dTcat -prefix average_spm fittsSPM_E8+orig[700..820]
3dMean -prefix average_raw temp_average_raw*
bash -c 'rm -f temp_average* &>/dev/null'


## 7. Extracting ROIs

In [10]:
# runAFNI('3dROIstats -mask ' + roi_path + ' CleanedData'+ID+'+orig.BRIK > ROIdata_' + ID + '.1D')
runAFNI('3dROIstats -mask ' + roi_path + ' CleanedData2'+ID+'+orig.BRIK > ROIdata_' + ID + '.1D')


3dROIstats -mask /home/aic/BigHDD/MR_DATA/CSI2018/template/ROI+orig CleanedDataE8+orig.BRIK > ROIdata_E8.1D
